In [3]:
pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow

     ------------------------------------ 272.8/272.8 MB 826.9 kB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 1.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 24.4/24.4 MB 180.3 kB/s eta 0:00:00
  Using cached numpy-1.23.5-cp39-cp39-win_amd64.whl (14.7 MB)
     -------------------------------------- 65.5/65.5 kB 147.3 kB/s eta 0:00:00
     ------------------------------------ 420.6/420.6 kB 234.5 kB/s eta 0:00:00
     ---------------------------------------- 4.1/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.4 which is incompatible.



     ---------------------------------------- 14.7/14.7 MB 1.1 MB/s eta 0:00:00
     -------------------------------------- 83.9/83.9 kB 943.9 kB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 696.8 kB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.8-py3-none-any.whl size=1439795 sha256=ca7ebbd9b2b3ed5d63ee72cfa6ea5f413bab10d2ebb97d13f1262016d91b2572
  Stored in directory: c:\users\john lorance\appdata\local\pip\cache\wheels\05\94\dc\81042da9bced43ff430bc02043d213d9e4b210b584c39e31c1
Successfully built jax
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


In [10]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=550bea9cf0967b7fe133161108c18d0ff28f9c5adf1ed0604c4b5d9097047f4a
  Stored in directory: c:\users\john lorance\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install opencv-python

     -------------------------------------- 38.2/38.2 MB 606.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [22]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [27]:
# load our serialized face detector model from disk
prototxtPath = r"E:\projects\Real time face mask detection\Face-Mask-Detection-master\face_detector\deploy.prototxt"
weightsPath = r"E:\projects\Real time face mask detection\Face-Mask-Detection-master\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [30]:
# load the face mask detector model from disk
maskNet = load_model("E:\projects\Real time face mask detection\Face-Mask-Detection-master\mask_detector.model")

In [31]:
# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

[INFO] starting video stream...


In [32]:
# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=400)

	# detect faces in the frame and determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		# determine the class label and color we'll use to draw
		# the bounding box and text
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

(1, 1, 200, 7)
1/1 [==============================] - 2s 2s/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 65ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 70ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/st

KeyboardInterrupt: 